In [14]:
%reload_ext dotenv
%dotenv .env

In [15]:
import os
import sqlite3
from collections import defaultdict

from classes import TramStop, TramPassage, DayType, Departure

connection = sqlite3.connect(os.environ.get('DATABASE_NAME'))
cursor = connection.cursor()

In [16]:
cursor.execute("SELECT id, name, latitude, longitude FROM tram_stops")

tram_stops: dict[int, TramStop] = {
    item[0]: TramStop(*item) for item in cursor.fetchall()
}

tram_stops

{213578731: TramStop(node_id=213578731, name='Kombinat 01', latitude=50.0791355, longitude=20.0650412),
 213585855: TramStop(node_id=213585855, name='Cienista 01', latitude=50.0853912, longitude=20.0385224),
 213585864: TramStop(node_id=213585864, name='Teatr Ludowy 01', latitude=50.082406, longitude=20.0329435),
 213585881: TramStop(node_id=213585881, name='Rondo Kocmyrzowskie im. Ks. Gorzelanego 02', latitude=50.0790506, longitude=20.0264899),
 213585893: TramStop(node_id=213585893, name='Bieńczycka 01', latitude=50.0755265, longitude=20.0201847),
 213585922: TramStop(node_id=213585922, name='Stella-Sawickiego 02', latitude=50.0743017, longitude=20.0046383),
 213585931: TramStop(node_id=213585931, name='AWF / PK 02', latitude=50.0740094, longitude=19.9989789),
 213592007: TramStop(node_id=213592007, name='Teatr Variété 02', latitude=50.058832, longitude=19.9623647),
 213593804: TramStop(node_id=213593804, name='Ofiar Dąbia 02', latitude=50.0596288, longitude=19.9762713),
 213593828: 

# Line 1

In [17]:
day, line_id, line_variant = DayType.WEEKDAY, 1, 1

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_line_variants.id = ? AND tram_departures.day = ?
    ORDER BY tram_departures.stop_index DESC,
             tram_departures.hour DESC,
             tram_departures.minute DESC
""", (line_variant, day.value))

stop_index_departures = defaultdict(list)
departures: list[Departure] = []

for item in cursor.fetchall():
    departure = Departure(item[0], line_variant, item[1], day, item[2], item[3])
    departures.append(departure)
    stop_index_departures[departure.stop_index].append(departure)

In [18]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(stop_index_departures.keys())
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second last stop somewhere else:", tram_stops[end_departure.stop_id].name)

        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = ? AND tram_line_variants.id = ? AND tram_departures.stop_index = ?
        """, (day.value, line_variant, end_departure.stop_index + 1,))

        new_stop_id = cursor.fetchone()[0]
        new_last_departure = Departure(
            new_stop_id,
            line_variant,
            end_departure.stop_index + 1,
            day,
            (end_departure.hour + (end_departure.minute + 1) // 60) % 24,
            (end_departure.minute + 1) % 60
        )

        departures.append(new_last_departure)
        passage.add_stop(new_last_departure)
        departure_passage[new_last_departure] = passage

    previous_stop = end_departure
    for stop_index in filter(
        lambda x: x <= end_departure.stop_index,
        sorted(stop_index_departures.keys(), reverse=True)
    ):
        next_stop = min(stop_index_departures[stop_index], key=lambda x: x.time_distance_between(previous_stop))

        if next_stop.time_distance_between(previous_stop) >= 60:
            break
        elif next_stop in departure_passage:
            later_passage = departure_passage[next_stop]
            for _ in range(later_passage.get_departure_index(next_stop), len(later_passage)):
                if later_passage.reverse_stop_sequence[-1].time_distance_between(previous_stop) < 100:
                    del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(next_stop)
        departure_passage[next_stop] = passage
        previous_stop = next_stop

[item.get_stop_names(tram_stops) for item in passages]

[['22:49 Elektromontaż 02',
  '22:50 Grodzki Urząd Pracy 02',
  '22:52 Wańkowicza 01',
  '22:53 Cienista 01',
  '22:54 Teatr Ludowy 01',
  '22:57 Rondo Kocmyrzowskie im. Ks. Gorzelanego 02',
  '22:58 Bieńczycka 01',
  '23:00 Rondo Czyżyńskie 04',
  '23:01 Centralna 02',
  '23:03 Rondo 308. Dywizjonu 02',
  '23:04 Ogród Doświadczeń 02',
  '23:05 TAURON Arena Kraków al. Pokoju 02',
  '23:08 Dąbie 02',
  '23:09 Ofiar Dąbia 02',
  '23:10 Fabryczna 02',
  '23:11 Francesco Nullo 02',
  '23:12 Teatr Variété 02',
  '23:15 Rondo Grzegórzeckie 02',
  '23:17 Hala Targowa 02',
  '23:18 Starowiślna 04',
  '23:20 Poczta Główna 02',
  '23:21 Plac Wszystkich Świętych 01',
  '23:23 Filharmonia 01',
  '23:25 UJ / AST 01',
  '23:26 Muzeum Narodowe 01',
  '23:27 Oleandry 01',
  '23:28 Park Jordana 01',
  '23:29 Reymana 01',
  '23:30 Cichy Kącik'],
 ['22:29 Elektromontaż 02',
  '22:30 Grodzki Urząd Pracy 02',
  '22:32 Wańkowicza 01',
  '22:33 Cienista 01',
  '22:34 Teatr Ludowy 01',
  '22:37 Rondo Kocmyrzo

## Line 50

#### Borek Fałęcki → Górka Narodowa P+R

In [19]:
day, line_id, line_variant = DayType.WEEKDAY, 21, 42

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_line_variants.id = ? AND tram_departures.day = ?
    ORDER BY tram_departures.stop_index DESC,
             tram_departures.hour DESC,
             tram_departures.minute DESC
""", (line_variant, day.value))

stop_index_departures = defaultdict(list)
departures: list[Departure] = []

for item in cursor.fetchall():
    departure = Departure(item[0], line_variant, item[1], day, item[2], item[3])
    departures.append(departure)
    stop_index_departures[departure.stop_index].append(departure)

In [20]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(stop_index_departures.keys())
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second last stop somewhere else:", tram_stops[end_departure.stop_id].name)

        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = ? AND tram_line_variants.id = ? AND tram_departures.stop_index = ?
        """, (day.value, line_variant, end_departure.stop_index + 1,))

        new_stop_id = cursor.fetchone()[0]
        new_last_departure = Departure(
            new_stop_id,
            line_variant,
            end_departure.stop_index + 1,
            day,
            (end_departure.hour + (end_departure.minute + 1) // 60) % 24,
            (end_departure.minute + 1) % 60
        )

        departures.append(new_last_departure)
        passage.add_stop(new_last_departure)
        departure_passage[new_last_departure] = passage

    previous_stop = end_departure
    for stop_index in filter(
        lambda x: x <= end_departure.stop_index,
        sorted(stop_index_departures.keys(), reverse=True)
    ):
        next_stop = min(stop_index_departures[stop_index], key=lambda x: x.time_distance_between(previous_stop))

        if next_stop.time_distance_between(previous_stop) >= 60:
            break
        elif next_stop in departure_passage:
            later_passage = departure_passage[next_stop]
            for _ in range(later_passage.get_departure_index(next_stop), len(later_passage)):
                if later_passage.reverse_stop_sequence[-1].time_distance_between(previous_stop) < 100:
                    del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(next_stop)
        departure_passage[next_stop] = passage
        previous_stop = next_stop

[item.get_stop_names(tram_stops) for item in passages]

[['22:56 Borek Fałęcki 01',
  '22:57 Borek Fałęcki I 02',
  '22:58 Solvay 02',
  '23:00 Łagiewniki SKA 03',
  '23:01 Sanktuarium Bożego Miłosierdzia 01',
  '23:03 Turowicza 01',
  '23:04 Kurdwanów P+R 03',
  '23:05 Kurdwanów P+R 01',
  '23:06 Witosa 02',
  '23:07 Nowosądecka 02',
  '23:08 Piaski Nowe 02',
  '23:10 Dauna 02',
  '23:12 Bieżanowska 04',
  '23:13 Kabel 02',
  '23:16 Dworzec Płaszów Estakada 01',
  '23:18 Lipska 02',
  '23:19 Gromadzka 02',
  '23:21 Kuklińskiego 02',
  '23:22 Klimeckiego 02',
  '23:23 Zabłocie 02',
  '23:27 Rondo Grzegórzeckie 03',
  '23:29 Rondo Mogilskie 05',
  '23:31 Dworzec Główny Tunel 02',
  '23:32 Politechnika 03',
  '23:34 Dworzec Towarowy 01',
  '23:36 Szpital Narutowicza 01',
  '23:37 Bratysławska 01',
  '23:38 Krowodrza Górka P+R 01',
  '23:40 Pachońskiego P+R 01',
  '23:41 Białoprądnicka 02',
  '23:42 Górnickiego 02',
  '23:43 Siewna Wiadukt 01',
  '23:44 Bociana 01',
  '23:45 Kuźnicy Kołłątajowskiej 01',
  '23:46 Papierni Prądnickich 01',
  '23

#### Górka Narodowa P+R → Borek Fałęcki

In [21]:
day, line_id, line_variant = DayType.WEEKDAY, 21, 41

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_line_variants.id = ? AND tram_departures.day = ?
    ORDER BY tram_departures.stop_index DESC,
             tram_departures.hour DESC,
             tram_departures.minute DESC
""", (line_variant, day.value))

stop_index_departures = defaultdict(list)
departures: list[Departure] = []

for item in cursor.fetchall():
    departure = Departure(item[0], line_variant, item[1], day, item[2], item[3])
    departures.append(departure)
    stop_index_departures[departure.stop_index].append(departure)

In [22]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(stop_index_departures.keys())
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second last stop somewhere else:", tram_stops[end_departure.stop_id].name)

        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = ? AND tram_line_variants.id = ? AND tram_departures.stop_index = ?
        """, (day.value, line_variant, end_departure.stop_index + 1,))

        new_stop_id = cursor.fetchone()[0]
        new_last_departure = Departure(
            new_stop_id,
            line_variant,
            end_departure.stop_index + 1,
            day,
            (end_departure.hour + (end_departure.minute + 1) // 60) % 24,
            (end_departure.minute + 1) % 60
        )

        departures.append(new_last_departure)
        passage.add_stop(new_last_departure)
        departure_passage[new_last_departure] = passage

    previous_stop = end_departure
    for stop_index in filter(
        lambda x: x <= end_departure.stop_index,
        sorted(stop_index_departures.keys(), reverse=True)
    ):
        next_stop = min(stop_index_departures[stop_index], key=lambda x: x.time_distance_between(previous_stop))

        if next_stop.time_distance_between(previous_stop) >= 60:
            break
        elif next_stop in departure_passage:
            later_passage = departure_passage[next_stop]
            for _ in range(later_passage.get_departure_index(next_stop), len(later_passage)):
                if later_passage.reverse_stop_sequence[-1].time_distance_between(previous_stop) < 100:
                    del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(next_stop)
        departure_passage[next_stop] = passage
        previous_stop = next_stop

[item.get_stop_names(tram_stops) for item in passages]

Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last stop somewhere else: Witosa 01
Second last

[['23:05 Górka Narodowa P+R 01',
  '23:06 Papierni Prądnickich 02',
  '23:07 Kuźnicy Kołłątajowskiej 02',
  '23:08 Bociana 02',
  '23:09 Siewna Wiadukt 02',
  '23:10 Górnickiego 01',
  '23:11 Białoprądnicka 01',
  '23:12 Pachońskiego P+R 02',
  '23:15 Krowodrza Górka P+R 02',
  '23:16 Bratysławska 02',
  '23:17 Szpital Narutowicza 02',
  '23:20 Dworzec Towarowy 02',
  '23:22 Politechnika 04',
  '23:23 Dworzec Główny Tunel 01',
  '23:25 Rondo Mogilskie 03',
  '23:28 Rondo Grzegórzeckie 04',
  '23:29 Zabłocie 01',
  '23:30 Klimeckiego 01',
  '23:32 Kuklińskiego 01',
  '23:33 Gromadzka 01',
  '23:35 Lipska 03',
  '23:37 Dworzec Płaszów Estakada 02',
  '23:39 Kabel 01',
  '23:41 Bieżanowska 03',
  '23:43 Dauna 01',
  '23:45 Piaski Nowe 01',
  '23:46 Nowosądecka 01',
  '23:48 Witosa 01',
  '23:49 Kurdwanów P+R 08',
  '23:50 Kurdwanów P+R 10',
  '23:51 Turowicza 02',
  '23:53 Sanktuarium Bożego Miłosierdzia 02',
  '23:55 Łagiewniki SKA 01',
  '23:56 Solvay 01',
  '23:57 Borek Fałęcki I 01',


## Line 4

In [23]:
day, line_id, line_variant = DayType.WEEKDAY, 3, 6

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_line_variants.id = ? AND tram_departures.day = ?
    ORDER BY tram_departures.stop_index DESC,
             tram_departures.hour DESC,
             tram_departures.minute DESC
""", (line_variant, day.value))

stop_index_departures = defaultdict(list)
departures: list[Departure] = []

for item in cursor.fetchall():
    departure = Departure(item[0], line_variant, item[1], day, item[2], item[3])
    departures.append(departure)
    stop_index_departures[departure.stop_index].append(departure)

In [24]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(stop_index_departures.keys())
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second last stop somewhere else:", tram_stops[end_departure.stop_id].name)

        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = ? AND tram_line_variants.id = ? AND tram_departures.stop_index = ?
        """, (day.value, line_variant, end_departure.stop_index + 1,))

        new_stop_id = cursor.fetchone()[0]
        new_last_departure = Departure(
            new_stop_id,
            line_variant,
            end_departure.stop_index + 1,
            day,
            (end_departure.hour + (end_departure.minute + 1) // 60) % 24,
            (end_departure.minute + 1) % 60
        )

        departures.append(new_last_departure)
        passage.add_stop(new_last_departure)
        departure_passage[new_last_departure] = passage

    previous_stop = end_departure
    for stop_index in filter(
        lambda x: x <= end_departure.stop_index,
        sorted(stop_index_departures.keys(), reverse=True)
    ):
        next_stop = min(stop_index_departures[stop_index], key=lambda x: x.time_distance_between(previous_stop))

        if next_stop.time_distance_between(previous_stop) >= 60:
            break
        elif next_stop in departure_passage:
            later_passage = departure_passage[next_stop]
            for _ in range(later_passage.get_departure_index(next_stop), len(later_passage)):
                if later_passage.reverse_stop_sequence[-1].time_distance_between(previous_stop) < 100:
                    del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(next_stop)
        departure_passage[next_stop] = passage
        previous_stop = next_stop

[item.get_stop_names(tram_stops) for item in passages]

[['23:11 Bronowice Małe 01',
  '23:12 Bronowice SKA 02',
  '23:14 Wesele 02',
  '23:15 Bronowice 02',
  '23:17 Głowackiego 02',
  '23:18 UKEN 02',
  '23:19 Biprostal 02',
  '23:20 Urzędnicza 02',
  '23:22 Plac Inwalidów 02',
  '23:23 Batorego 02',
  '23:25 Teatr Bagatela 03',
  '23:27 Stary Kleparz 02',
  '23:29 Teatr Słowackiego 02',
  '23:31 Lubicz 02',
  '23:33 Rondo Mogilskie 02',
  '23:34 Cystersów 01',
  '23:36 Białucha 01',
  '23:38 TAURON Arena Kraków Wieczysta 01',
  '23:39 Muzeum Lotnictwa Polskiego 01',
  '23:41 AWF / PK 01',
  '23:42 Stella-Sawickiego 01',
  '23:43 Czyżyny 01',
  '23:46 Rondo Czyżyńskie 01',
  '23:47 Os. Kolorowe 01',
  '23:50 Plac Centralny im. R. Reagana 03',
  '23:52 Struga 01',
  '23:55 Kombinat 01',
  '23:56 Zajezdnia Nowa Huta 01'],
 ['22:47 Bronowice Małe 01',
  '22:48 Bronowice SKA 02',
  '22:50 Wesele 02',
  '22:51 Bronowice 02',
  '22:53 Głowackiego 02',
  '22:54 UKEN 02',
  '22:55 Biprostal 02',
  '22:56 Urzędnicza 02',
  '22:58 Plac Inwalidów 02

## Line 20

In [25]:
day, line_id, line_variant = DayType.WEEKDAY, 15, 30

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_line_variants.id = ? AND tram_departures.day = ?
    ORDER BY tram_departures.stop_index DESC,
             tram_departures.hour DESC,
             tram_departures.minute DESC
""", (line_variant, day.value))

stop_index_departures = defaultdict(list)
departures: list[Departure] = []

for item in cursor.fetchall():
    departure = Departure(item[0], line_variant, item[1], day, item[2], item[3])
    departures.append(departure)
    stop_index_departures[departure.stop_index].append(departure)

In [26]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(stop_index_departures.keys())
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second last stop somewhere else:", tram_stops[end_departure.stop_id].name)

        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = ? AND tram_line_variants.id = ? AND tram_departures.stop_index = ?
        """, (day.value, line_variant, end_departure.stop_index + 1,))

        new_stop_id = cursor.fetchone()[0]
        new_last_departure = Departure(
            new_stop_id,
            line_variant,
            end_departure.stop_index + 1,
            day,
            (end_departure.hour + (end_departure.minute + 1) // 60) % 24,
            (end_departure.minute + 1) % 60
        )

        departures.append(new_last_departure)
        passage.add_stop(new_last_departure)
        departure_passage[new_last_departure] = passage

    previous_stop = end_departure
    for stop_index in filter(
        lambda x: x <= end_departure.stop_index,
        sorted(stop_index_departures.keys(), reverse=True)
    ):
        next_stop = min(stop_index_departures[stop_index], key=lambda x: x.time_distance_between(previous_stop))

        if next_stop.time_distance_between(previous_stop) >= 60:
            break
        elif next_stop in departure_passage:
            later_passage = departure_passage[next_stop]
            for _ in range(later_passage.get_departure_index(next_stop), len(later_passage)):
                if later_passage.reverse_stop_sequence[-1].time_distance_between(previous_stop) < 100:
                    del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(next_stop)
        departure_passage[next_stop] = passage
        previous_stop = next_stop

[item.get_stop_names(tram_stops) for item in passages]

[['23:07 Cichy Kącik 01',
  '23:08 Reymana 02',
  '23:09 Park Jordana 02',
  '23:10 Oleandry 02',
  '23:12 Muzeum Narodowe 02',
  '23:14 UJ / AST 02',
  '23:16 Teatr Bagatela 01',
  '23:18 Stary Kleparz 02',
  '23:20 Teatr Słowackiego 02',
  '23:22 Lubicz 02',
  '23:24 Rondo Mogilskie 03',
  '23:27 Rondo Grzegórzeckie 04',
  '23:28 Zabłocie 01',
  '23:29 Klimeckiego 01',
  '23:31 Kuklińskiego 01',
  '23:32 Gromadzka 01',
  '23:34 Lipska 01',
  '23:35 Rzebika 01',
  '23:36 Mały Płaszów P+R'],
 ['22:47 Cichy Kącik 01',
  '22:48 Reymana 02',
  '22:49 Park Jordana 02',
  '22:50 Oleandry 02',
  '22:52 Muzeum Narodowe 02',
  '22:54 UJ / AST 02',
  '22:56 Teatr Bagatela 01',
  '22:58 Stary Kleparz 02',
  '23:00 Teatr Słowackiego 02',
  '23:02 Lubicz 02',
  '23:04 Rondo Mogilskie 03',
  '23:07 Rondo Grzegórzeckie 04',
  '23:08 Zabłocie 01',
  '23:09 Klimeckiego 01',
  '23:11 Kuklińskiego 01',
  '23:12 Gromadzka 01',
  '23:14 Lipska 01',
  '23:15 Rzebika 01',
  '23:16 Mały Płaszów P+R'],
 ['22: